In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import ChunkedEncodingError
import time

In [ ]:
### Due to the html size is too big, converted to txt file for easy processing with BeautifulSoup
f1  =  open("index.txt" , "r")
soup = BeautifulSoup(f1)

In [ ]:
type(soup)

In [ ]:
all_company_info = soup.find_all(name="div", class_="_WfdmOZ2wLBd0x_TFyKO")

In [ ]:
company_list = []
company_url_list = []
company_rating = []

for i in range(len(all_company_info)):
    company_list.append(all_company_info[i].find(name="a", class_="_9d6_WXgVpZWn7HaXEgs").getText())
    
## Get company jobstreet main url
endpoint = "https://www.jobstreet.com.my"
for i in range(len(all_company_info)):
    company_url_list.append(endpoint + all_company_info[i].find(name="a", class_="_9d6_WXgVpZWn7HaXEgs")["href"])


for i in range(len(all_company_info)):
    company_rating.append(float(all_company_info[i].find_all(name="span", class_="j6BF91AmxWn9vS_m399j")[1].getText().split("out")[0]))

In [ ]:
## Convert them into the basic dataframe
data = {
    "company_name" : company_list,
    "company_js_url" : company_url_list,
    "company_rating" : company_rating
}

df = pd.DataFrame(data=data)

df.head()

In [ ]:
## Remove company without rating
df = df[df["company_rating"] != 0]

In [ ]:
## List of info
five_star_list = []
four_star_list = []
three_star_list = []
two_star_list = []
one_star_list = []
salary_rate_list = []
recommendation_rate_list = []

## To extract more information from the website
for url in df["company_js_url"]:
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, "html.parser")

    ## Info in the response
    five_star = soup.find_all(name="span", class_="DCaw56gNq5O1IBoC5xcz")[0].getText()
    four_star = soup.find_all(name="span", class_="DCaw56gNq5O1IBoC5xcz")[1].getText()
    three_star = soup.find_all(name="span", class_="DCaw56gNq5O1IBoC5xcz")[2].getText()
    two_star = soup.find_all(name="span", class_="DCaw56gNq5O1IBoC5xcz")[3].getText()
    one_star =  soup.find_all(name="span", class_="DCaw56gNq5O1IBoC5xcz")[4].getText()
    salary_rate = soup.find_all(name="div", class_="RL2EfHhuwPsO8xlgwG_S")[0].getText()
    recommendation_rate = soup.find_all(name="div", class_="RL2EfHhuwPsO8xlgwG_S")[1].getText()

    try:
        salary_rate_list.append(salary_rate)
    except:
        salary_rate_list.append("No Info")

    try:
        recommendation_rate_list.append(recommendation_rate)
    except:
        recommendation_rate_list.append("No Info")

    try:
        five_star_list.append(five_star)
    except:
        five_star_list.append("0")

    try:
        four_star_list.append(four_star)
    except:
        four_star_list.append("0")

    try:
        three_star_list.append(three_star)
    except:
        three_star_list.append("0")

    try:
        two_star_list.append(two_star)
    except:
        two_star_list.append("0")

    try:
        one_star_list.append(one_star)
    except:
        one_star_list.append("0")

In [ ]:
## Save to the dataframe
df["five_star_rating"] = five_star_list
df["four_star_rating"] = four_star_list
df["three_star_rating"] = three_star_list
df["two_star_rating"] = two_star_list
df["one_star_rating"] = one_star_list
df["salary_rate"] = salary_rate_list
df["recommendation_rate"] = recommendation_rate_list

In [ ]:
## Function created to extract all the extra info from each company jobstreet website url
## To unfilter out no info in later stage
extra_info_list = []

for i in df["company_js_url"]:
    response = requests.get(i)
    data = response.text
    soup = BeautifulSoup(data, "html.parser")

    try:
        extra_info_list.append(soup.find(name="div", class_="E5REvPGaM2o9YsLBgK75").getText())
    except:
        extra_info_list.append("No Info")

In [ ]:
## Filter data with No Info
df = df[df["extra_info"] != "No Info"]

In [ ]:
## Create a list of lists for more info
address_list = []
website_list = []
employee_num_list = []
benefit_list = []
dress_code_list = []
language_list = []
working_hours = []



all_company_info_list = []
c = 0

for url in df_without_no_info["company_js_url"]:
    
    ## Function to avoid connection peer error
    trycnt = 3  # max try cnt
    while trycnt > 0:
        try:
            result = requests.get(url)
            c += 1
            trycnt = 0 # success
        except ChunkedEncodingError as ex:
            if trycnt <= 0: print("Failed to retrieve: " + url + "\n" + str(ex))  # done retrying
            else: trycnt -= 1  # retry
            time.sleep(0.5)  # wait 1/2 second then retry
    
    ## Call each company website url and extract the function to single dimensional array
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, "html.parser")
    individual_company_info = []
    cleaned_individual_company_info = []

    for i in soup.find_all(name="div", class_="kAiAILUw3ZWmw8LarQq0"):
        individual_company_info.append(i.getText())
    
    location = [s for s in individual_company_info if "Location" in s]
    cleaned_individual_company_info.append(location[0] if location else "No Info")

    website = [s for s in individual_company_info if "Website" in s]
    cleaned_individual_company_info.append(website[0] if website else "No Info")

    company_size = [s for s in individual_company_info if "Company Size" in s]
    cleaned_individual_company_info.append(company_size[0] if company_size else "No Info")

    benefit = [s for s in individual_company_info if "Benefits" in s]
    cleaned_individual_company_info.append(benefit[0] if benefit else "No Info")

    dress_code = [s for s in individual_company_info if "Dress code" in s]
    cleaned_individual_company_info.append(dress_code[0] if dress_code else "No Info")

    spoken_language = [s for s in individual_company_info if "Spoken language" in s]
    cleaned_individual_company_info.append(spoken_language[0] if spoken_language else "No Info")

    work_hour = [s for s in individual_company_info if "Work hour" in s]
    cleaned_individual_company_info.append(work_hour[0] if work_hour else "No Info")

    all_company_info_list.append(cleaned_individual_company_info)

In [ ]:
## Save the collected info to list by list comprehension function
address_list = [i[0] for i in all_company_info_list]
website_list = [i[1] for i in all_company_info_list]
employee_num_list = [i[2] for i in all_company_info_list]
benefit_list = [i[3] for i in all_company_info_list]
dress_code_list = [i[4] for i in all_company_info_list]
language_list = [i[5] for i in all_company_info_list]
working_hours = [i[6] for i in all_company_info_list]

In [ ]:
## Drop the extra info column which we no longer need to use
df = df.drop("extra_info", index=1)

In [ ]:
## Futher cleaning on the dataset
df["company_address"] = df["company_address"].apply(lambda x:x.replace("Location", "").replace('\n', ' ').replace('\r', '').strip())
df["company_website"] = df["company_website"].apply(lambda x:x.replace("Website", "").replace('\n', ' ').replace('\r', '').strip())
df["company_employee_num"] = df["company_employee_num"].apply(lambda x:x.replace("Company Size", "").replace('\n', ' ').replace('\r', '').strip())
df["company_benefit"] = df["company_benefit"].apply(lambda x:x.replace("Benefits", "").replace('\n', ' ').replace('\r', '').strip())
df["company_dress_code"] = df["company_dress_code"].apply(lambda x:x.replace("Dress code", "").replace('\n', ' ').replace('\r', '').strip())
df["company_language"] = df["company_language"].apply(lambda x:x.replace("Spoken language", "").replace('\n', ' ').replace('\r', '').strip())
df["company_work_hours"] = df["company_work_hours"].apply(lambda x:x.replace("Work hours", "").replace('\n', ' ').replace('\r', '').strip())

In [ ]:
## Save to CSV
df.to_csv("company_list_with_info.csv")